In [1]:
def pipeline_decision(params):
    actions = []

    if params["branch"] == "main":
        if params["changed_files"] > 100:
            actions.append("skip build")  # Line A: Skip build for large changes on main
        else:
            actions.append("build")  # Line B: Build main branch

        if params["dependency_stability"] == "unstable":
            actions.append("run integration tests")  # Line C: Run integration tests if dependencies are unstable
        elif params["dependency_stability"] == "stable" and params["test_results"] == "pass":
            actions.append("deploy")  # Line D: Deploy to production if main and stable dependencies

    elif params["branch"].startswith("feature/"):
        actions.append("build")  # Line E: Build feature branch

        if params["changed_files"] < 50 and params["review_status"] == "approved":
            actions.append("run unit tests")  # Line F: Run tests on small, approved feature branches

    if params["critical_bug_found"]:
        actions.append("halt deployment")  # Line G: Halt if critical bug

    if params["last_deployment_status"] == "failure" and params["branch"] == "main":
        actions.append("notify team")  # Line H: Notify team on repeated failure in main branch

    return actions

In [35]:
from debugging_framework.fuzzingbook.grammar import is_valid_grammar

pipeline_grammar = {
    "<start>": ["<pipeline_input>"],
    "<pipeline_input>": ["{ <branch>, <changed_files>, <dependency_stability>, <test_results>, <review_status>, <critical_bug_found>, <last_deployment_status> }"],
    
    "<branch>": ["'branch': <main_branch>", "'branch': <feature_branch>"],
    "<main_branch>": ["'main'"],
    "<feature_branch>": ["'feature/<feature_name>'"],
    "<feature_name>": ["<alphanum>", "<alphanum><feature_name>"],
    
    "<changed_files>": ["'changed_files': <nonnegative_integer>"],
    
    "<dependency_stability>": ["'dependency_stability': 'stable'", "'dependency_stability': 'unstable'"],
    
    "<test_results>": ["'test_results': 'pass'", "'test_results': 'fail'"],
    
    "<review_status>": ["'review_status': 'approved'", "'review_status': 'pending'", "'review_status': 'rejected'"],
    
    "<critical_bug_found>": ["'critical_bug_found': True", "'critical_bug_found': False"],
    
    "<last_deployment_status>": ["'last_deployment_status': 'success'", "'last_deployment_status': 'failure'"],
    
    "<nonnegative_integer>": ["<digit>", "<digit><nonnegative_integer>"],
    
    "<alphanum>": ["<letter>", "<digit>"],
    "<letter>": [chr(letter) for letter in range(ord('a'), ord('z') + 1)] + [chr(letter) for letter in range(ord('A'), ord('Z') + 1)],
    "<digit>": [str(digit) for digit in range(10)]
}

is_valid_grammar(pipeline_grammar)

True

In [36]:
from debugging_framework.fuzzingbook.fuzzer import GrammarFuzzer

fuzzer = GrammarFuzzer(pipeline_grammar)

for _ in range(5):
    inp = fuzzer.fuzz()
    print(inp)
    print(inp, pipeline_decision(eval(inp)))

{ 'branch': 'main', 'changed_files': 5, 'dependency_stability': 'unstable', 'test_results': 'fail', 'review_status': 'approved', 'critical_bug_found': True, 'last_deployment_status': 'failure' }
{ 'branch': 'main', 'changed_files': 5, 'dependency_stability': 'unstable', 'test_results': 'fail', 'review_status': 'approved', 'critical_bug_found': True, 'last_deployment_status': 'failure' } ['build', 'run integration tests', 'halt deployment', 'notify team']
{ 'branch': 'feature/2p', 'changed_files': 9, 'dependency_stability': 'stable', 'test_results': 'fail', 'review_status': 'pending', 'critical_bug_found': True, 'last_deployment_status': 'success' }
{ 'branch': 'feature/2p', 'changed_files': 9, 'dependency_stability': 'stable', 'test_results': 'fail', 'review_status': 'pending', 'critical_bug_found': True, 'last_deployment_status': 'success' } ['build', 'halt deployment']
{ 'branch': 'main', 'changed_files': 80, 'dependency_stability': 'stable', 'test_results': 'fail', 'review_status': 

In [2]:
params1 = {
    "branch": "main",
    "changed_files": 150,
    "dependency_stability": "stable",
    "test_results": "pass",
    "review_status": "approved",
    "critical_bug_found": False,
    "last_deployment_status": "success"
}

In [3]:
pipeline_decision(params1)

['skip build', 'deploy']

In [4]:
params1 = """{
    "branch": "main",
    "changed_files": 20,
    "dependency_stability": "stable",
    "test_results": "pass",
    "review_status": "approved",
    "critical_bug_found": False,
    "last_deployment_status": "success"
}"""

In [15]:
params2 = """{
    "branch": "main",
    "changed_files": 150,
    "dependency_stability": "stable",
    "test_results": "pass",
    "review_status": "approved",
    "critical_bug_found": True,
    "last_deployment_status": "success"
}"""

In [16]:
from linetracer import CoveragePyAnalyzer
from pathlib import Path

In [17]:
analyzer = CoveragePyAnalyzer(
    project_root=Path.cwd() / "pipeline",
    harness=Path.cwd() / "harness.py",
    test=[params2])

report = analyzer.get_coverage()
print(report)
print(report.coverage_data)

trigg = dict()
for file, lines in report.total_executable_lines.items():
    trigg[file] = lines.intersection(report.coverage_data[file])

print(trigg)
su = 0
for file, lines in trigg.items():
    print(f"{file}: {len(lines)}")
    su += len(lines)

print(f"Total: {su}")
print(f"Coverage: {su / sum(len(lines) for lines in report.total_executable_lines.values())}")

['skip build', 'deploy', 'halt deployment']

Name                   Stmts   Miss  Cover
------------------------------------------
pipeline/__init__.py       0      0   100%
pipeline/pipeline.py      21      7    67%
------------------------------------------
TOTAL                     21      7    67%
66.66666666666667
CoverageReport(['pipeline.py: {1, 2, 4, 5, 6, 10, 12, 13, 21, 22, 24, 27, 30, 31}', '__init__.py: {0}'], coverage=0.7142857142857143)
{'/Users/martineberlein/github/avicenna/notebooks/pipeline/pipeline.py': {1, 2, 4, 5, 6, 10, 12, 13, 21, 22, 24, 27, 30, 31}, '/Users/martineberlein/github/avicenna/notebooks/pipeline/__init__.py': {0}}
{'/Users/martineberlein/github/avicenna/notebooks/pipeline/pipeline.py': {1, 2, 4, 5, 6, 10, 12, 13, 21, 22, 24, 27, 30, 31}, '/Users/martineberlein/github/avicenna/notebooks/pipeline/__init__.py': set()}
/Users/martineberlein/github/avicenna/notebooks/pipeline/pipeline.py: 14
/Users/martineberlein/github/avicenna/notebooks/pipeline/__init_

In [21]:
12 in report.coverage_data['/Users/martineberlein/github/avicenna/notebooks/pipeline/pipeline.py']

True